In [1]:
###imports###
##pip install python-binance to use the packages below###

from binance.client import Client
from binance.websockets import BinanceSocketManager
from binance.exceptions import BinanceAPIException, BinanceOrderException

from twisted.internet import reactor

import numpy as np 
import pytz
from datetime import timezone
from datetime import datetime
from time import sleep

import pandas as pd
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
##see https://testnet.binance.vision/ to sign up for a demo account###

api_key_demo = ""
api_secret_demo = ""

In [ ]:
##get account balance/wallet###

client = Client(api_key_demo, api_secret_demo)

client.API_URL = 'https://testnet.binance.vision/api'

print(client.get_account())

In [ ]:
###create skeleton of dataframe to add features later (during the connection)###

price = {"XRPUSDT": pd.DataFrame(columns=['date', 'open','price', 'high', 'low', 'volume','ma','ma2', 'bl', 'value']), "TRXUSDT": pd.DataFrame(columns=['date', 'open','price','high', 'low', 'volume','ma','ma2', 'bl', 'value']), 'LTCUSDT': pd.DataFrame(columns=['date','open', 'price','high', 'low', 'volume','ma','bu', 'bl', 'value']), 'error':False}

def btc_pairs_trade(msg):
    if msg['e'] != 'error':
        price["XRPUSDT"].loc[len(price["XRPUSDT"])] = [pd.Timestamp.now(),float(msg['o']), float(msg['c']),float(msg['h']), float(msg['l']), float(msg['v']), np.nan, np.nan, np.nan, np.nan]
    else:
        price['error']:True
            
            
def btc_pairs_trade2(msg):
    if msg['e'] != 'error':
        price["TRXUSDT"].loc[len(price["TRXUSDT"])] = [pd.Timestamp.now(), float(msg['o']), float(msg['c']), float(msg['h']), float(msg['l']),float(msg['v']), np.nan, np.nan, np.nan, np.nan]
    else:
        price['error']:True

In [ ]:
###initial portfolio value and current crypto balances###

x = client.get_account()

xrp_i = client.get_symbol_ticker(symbol="XRPUSDT")
trx_i = client.get_symbol_ticker(symbol="TRXUSDT")

print('Initial Price of TRX: ' + str(float(trx_i['price'])))
print('Initial Price of XRP: ' + str(float(xrp_i['price'])))
print('')

balance = float(x['balances'][6]['free'])
trx = float(x['balances'][5]['free'])
xrp = float(x['balances'][7]['free'])

print( 'Starting Account Balance: ' + str(x['balances'][6]['free']))
print( 'Starting TRX Balance: ' + str(x['balances'][5]['free']))
print( 'Starting XRP Balance: ' + str(x['balances'][7]['free']))

print('')

value = balance + float(xrp_i['price'])*xrp + float(trx_i['price'])*trx

print('Initial Approximate Portfolio value: ' + str(value))

In [ ]:
def create_order():
    
    bsm = BinanceSocketManager(client)
    conn_key = bsm.start_symbol_ticker_socket("XRPUSDT", btc_pairs_trade2)
    conn_key2 = bsm.start_symbol_ticker_socket("TRXUSDT", btc_pairs_trade)
    bsm.start()
    
    z = client.get_account()
    
    while len(price["XRPUSDT"])  == 0:
        sleep(0.1)
    
    sleep(400)
    
    stocks = ["XRPUSDT", "TRXUSDT"]
    
    for stock in stocks:
    
        df = price[stock]
        
        try:
            
            ###input algorithm here - cond_1:cond_n that need to be satisfied for a buy/sell##
            
            df = df.sort_values('date', ascending = False)

            
            if ( (cond_1 & cond_4)| (cond_2 & ~cond_3)  )&(stock == 'XRPUSDT'):
                order = client.order_market_sell(symbol=stock, quantity=25)
                
            elif ( (cond_1 & cond_4)| (cond_2 & ~cond_3)  )&(stock == "TRXUSDT"):
                order = client.order_market_sell(symbol=stock, quantity=300)

            elif ((cond_1 & cond_3)| (cond_2 & cond_4) )&(stock == "TRXUSDT"):
                order = client.order_market_buy(symbol=stock, quantity=300)
                
            elif ( (cond_1 & cond_3)| (cond_2 & cond_4) )&(stock == 'XRPUSDT'):
                order = client.order_market_buy(symbol=stock, quantity=25)
                
            else:
                print("Did not Buy")

            sleep(0.1)
            
        except:
            print('An Error Occured...')
    
    h = client.get_account()
    
    print(' ')
    now = datetime.now()
    dt_string = now.strftime("%H:%M:%S")
    print("Time: ", dt_string)
    print( 'Difference Account Balance: ' + str(float(h['balances'][6]['free']) - float(z['balances'][6]['free'])))
    print( 'Difference TRX Balance: ' + str(float(h['balances'][5]['free']) - float(z['balances'][5]['free'])))
    print( 'Difference XRP Balance: ' + str(float(h['balances'][7]['free']) - float(z['balances'][7]['free'])))
    print(' ')
    print('Next Iteration:')
    print(' ')

In [ ]:
while True:
    
    create_order()

    sleep(250)

In [ ]:
###get balances and portfolio value after algo is finished###

y = client.get_account()

xrp_f = client.get_symbol_ticker(symbol="XRPUSDT")
trx_f = client.get_symbol_ticker(symbol="TRXUSDT")

print('Final Price of TRX: ' + str(float(trx_f['price'])))
print('Final Price of XRP: ' + str(float(xrp_f['price'])))
print('')

balance = float(y['balances'][6]['free'])
trx_final = float(y['balances'][5]['free'])
xrp_final = float(y['balances'][7]['free'])

print( 'Ending Account Balance: ' + str(y['balances'][6]['free']))
print( 'Ending TRX Balance: ' + str(y['balances'][5]['free']))
print( 'Ending XRP Balance: ' + str(y['balances'][7]['free']))

print('')

value_final = balance + float(xrp_f['price'])*xrp_final + float(trx_f['price'])*trx_final

change = 100*(float(y['balances'][6]['free']) - float(x['balances'][6]['free']))/float(x['balances'][6]['free'])
change_xrp = 100*(float(y['balances'][7]['free']) - float(x['balances'][7]['free']))/float(x['balances'][7]['free'])
change_trx = 100*(float(y['balances'][5]['free']) - float(x['balances'][5]['free']))/float(x['balances'][5]['free'])
change_final = 100*((value_final - value)/value)

print(' ')
print(' ')
print( '% Difference Account Balance: ' + str(change) +'%')
print( '% Difference XRP: ' + str(change_xrp) +'%')
print( '% Difference TRX: ' + str(change_trx) +'%')
print(' ')
print( '% Difference Portfolio Value: ' + str(change_final) +'%')
print(' ')
print('Final Approximate Portfolio Value: ' + str(value_final))